In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output
import base64

import numpy as np
import pandas as pd


from pymongo import MongoClient



###########################
# Data Manipulation / Model
###########################
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD import AnimalShelter
# FIX ME update with your username and password and CRUD Python module name
username = "aacuser"
password = "Nafepafe1230"
shelter = AnimalShelter()
shelter.initialize(username, password)



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# readAll is the method name from AnimalShelter class and returns a cursor object
# update the next line to match your method name from your CRUD Python module class
df = pd.DataFrame.from_records(shelter.read({}))

animal_types = df['animal_type'].unique().tolist()


#Add in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png' # customer image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image', width = 200),

app.layout = html.Div([
    html.Div(
        html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image', width = 200),
        style={'textAlign': 'center'}
    ),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    
    html.Div([
        html.Label("Filter by:"),
        dcc.Dropdown(
            id = "filter_by",
            options=[{'label':i, 'value':i} for i in animal_types],
            value = 'Dog',
            clearable = False)
        ]),
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data = df.to_dict('records'),
        editable = False,
        #Set up the features for your interactive data table to make it user-friendly for your client
         #filter_action = "native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(
        id='map-id'
    ),
    html.Div(
        id='graph-id'

    ),
    html.Center(html.B(html.H1("Nathan Jantz Project 2")))
])

@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter_by', 'value')]
)
def set_filter(sort_val):
    df = pd.DataFrame(list(shelter.read({"animal_type":str(sort_val)})))
    return df.to_dict('records')

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style = {'left': '250px', 'height': '750px', 'width': '1500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]

@app.callback(
    Output('graph-id', "children"),
    [Input('filter_by', 'value')]
)
def update_graph(filter_val):
    df3 = pd.DataFrame(list(shelter.read({"animal_type":str(filter_val)})))
    animal_breeds = df3['breed'].unique().tolist()
    fig =  px.pie(df3, values = df3['breed'].value_counts(), names = animal_breeds)
    return [
        dcc.Graph(style = {'left': '250px', 'height': '1000px', 'width': '1500px'},
            figure = fig
        )
    ]

app
